# C++ Introduction - Classes

### Terms
- **mutator**: a method that mutates (changes) class state
- **accessor**: a method that returns class state (should typically be declared `const`)
- **private helper**: a `private` method (and thus that can only be called within the class)

In [1]:
#include <string>
#include <iostream>
#include <sstream>
using namespace std;

In [2]:
/*
  This function prints out all the bytes of the thing you pass in.
  At some point you'll know about all the parts of this function,
    but if you don't yet, that's ok.
*/
template <class T> void printBytes(T const& thing) {
    char* addr = (char*)&thing;
    for (int i = 0; i < sizeof(T); i++) {
        printf("%02X ", addr[i]);
    }
    printf("\n");
}

## `sizeof`

In [3]:
int number;

In [4]:
printBytes(number);

00 00 00 00 


In [5]:
number = 7;
printBytes(number);

07 00 00 00 


In [6]:
number = 1234567890;
printBytes(number);

D2 02 96 49 


In [7]:
sizeof(int)

4

## `struct`

In [8]:
struct TwoInts {
    int first;
    int second;
}

In [9]:
TwoInts foo;
TwoInts bar;

In [10]:
sizeof(TwoInts)

8

In [11]:
printBytes(foo);

00 00 00 00 00 00 00 00 


In [12]:
foo.first = 255;
foo.second = 7;
printBytes(foo)

FF 00 00 00 07 00 00 00 


A **struct** is a simple structuring of memory. It packs the member variables next to each other and keeps track of which is which.

Just like in Python, `.` is the **member access operator**. 

## `class`

In [13]:
class TwoNumbers {
    double first;
    double second;
}

In [14]:
sizeof(double)

8

In [15]:
sizeof(TwoNumbers)

16

In [16]:
TwoNumbers doubles;
printBytes(doubles)

00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 


In [17]:
TwoNumbers doubles;
doubles.first = 1.234e134;
printBytes(doubles)

input_line_30:3:9: error: 'first' is a private member of '__cling_N515::TwoNumbers'
doubles.first = 1.234e134;
        ^
input_line_24:2:12: note: implicitly declared private here
    double first;
           ^


Interpreter Error: 

Members of a **class** are **private** by default.

Members of a **struct** are **public** by default.

Private members cannot by accessed by anything but the object itself (or declared *friends* of the class).

Public members are accessible by everyone.

In [18]:
class TwoNumbers {
    public:
    double first;
    double second;
}

In [19]:
TwoNumbers doubles;
doubles.first = 1.234e134;
doubles.second = 1.0;
printBytes(doubles)

4B BC A8 F3 3A BB C5 5B 00 00 00 00 00 00 F0 3F 


## Member Access

In [20]:
TwoNumbers* moreNumbers = new TwoNumbers;

In [21]:
moreNumbers.first = 7.1234556

input_line_34:2:13: error: member reference type '__cling_N522::TwoNumbers *' is a pointer; did you mean to use '->'?
 moreNumbers.first = 7.1234556
 ~~~~~~~~~~~^
            ->


Interpreter Error: 

In [22]:
(*moreNumbers).first = 7.12345;

In [23]:
moreNumbers->second = 2346.080809;

In [24]:
class TwoNumbers {
    private:
    double first;
    double second;
    public:
    double getFirst() { return this->first; }
    double getSecond() { return this->second; }
}

What is `this`?

A pointer to the object instance!

`this` is effectively the same thing as `self` in Python.

In [25]:
class GPS {
    public:
    void whereAmI() {
        printf("%p\n", this);
    }
}

In [26]:
GPS gps;
gps.whereAmI();
printf("%p\n", &gps);

0xffff8bb09068
0xffff8bb09068


## Constructors

In [ ]:
class TwoNumbers {
    private:
    double first;
    double second;
    public:
    double getFirst() { return this->first; }
    double getSecond() { return this->second; }
}

How do I set the values for `first` and `second`?

<div class='big centered'>🤨</div>

A pattern you will often see (especially in C++), is to make member variables private, but then provide **setter** methods to access them.

In [27]:
class HasADouble {
    private:
    double cantTouchThis;
    
    public:
    void setDouble(double const& value) {
        this->cantTouchThis = value;
    }
    double getDouble() {
        return this->cantTouchThis;
    }
}

In [28]:
HasADouble hasDouble;
hasDouble.setDouble(3.14);
cout << hasDouble.getDouble() << endl;

3.14


But if you're going to make setter and getter methods, why not make the variable public?

<div class='big centered'>🤔 🤨 🤪 🫠</div>

So instead, avoid setter methods and instantiate everything in the constructor.

When you instantiate a class, the compiler calls the class constructor.

The **constructor** is very similar to the **`__init__`** method in python.

The constructor is *always* named after the class name (required by compiler). 

If you don't specify a constructor in your class declaration, C++ makes one for you.

The *default constructor* is a no-argument, no-logic method. E.g.:

```
HasADouble() {}
```

The constructor's job is to make sure all the member variables of the class are initialized correctly.

In [29]:
class HasADouble2 {
    private:
    double cantTouchThis;
    
    public:
    HasADouble2(double const& value) {  // Constructor
        this->cantTouchThis = value;
    }
    
    double getDouble() {
        return this->cantTouchThis;
    }
}

In [30]:
HasADouble2 hasDouble(3.14);
cout << hasDouble.getDouble() << endl;

3.14


When all the constructor needs to do is assign values, then you should use the **initializer list** instead.

In [32]:
class HasADouble3 {
    private:
    double cantTouchThis;
    
    public:
    HasADouble3(double const& value) : cantTouchThis(value) { }
    
    double getDouble() const {
        return this->cantTouchThis;
    }
}

Use the initializer list to initialize member variables.

Put any other necessary logic in the constructor. It's OK for it to be empty.

Now, consider the following two blocks of code. 

Let's assume it is **really** important that the object be initialized with a value before it is used.

In [33]:
HasADouble first;
first.setDouble(3.14);
cout << first.getDouble() << endl;

3.14


Is it possible to get this example right?

I.e. can you make sure that `setDouble` is called before you do anything else?

Can you get it wrong?

In [34]:
HasADouble3 second(3.14159);
cout << second.getDouble() << endl;

3.14159


Is it possible to get this example right?

I.e. can you make sure that the object is instantiated correctly?

Can you get it wrong?

<div class='alert alert-info'>The question is not <br/><br/> 
    <div style='font-size: 24px'><i>Can I get it right?</i></div>
    <br/><br/> 
    but rather <br/><br/> 
    <div style='font-size: 48px'><b>Is it impossible to get wrong?</b></div>
</div>

<div class='alert alert-info'>If you write code you can't mess up, you won't mess it up.</div>

<div class='big centered'>🧐</div>

In [35]:
class Noisy {
    public:
    Noisy() {
        cout << "A new Noisy has come into existence!" << endl;
    }
}

In [36]:
Noisy noisy;

A new Noisy has come into existence!


## Destructors

In [37]:
class Noisy {
    public:
    Noisy() {
        cout << "A new Noisy has come into existence!" << endl;
    }
    ~Noisy() {
        cout << "One more Noisy is departing this world. Adieu." << endl;
    }
}

In [38]:
void doSomethingNoisy() {
    Noisy noisy;
}

In [39]:
doSomethingNoisy()

A new Noisy has come into existence!
One more Noisy is departing this world. Adieu.


In [40]:
Noisy* noisy = new Noisy;

A new Noisy has come into existence!


In [41]:
delete noisy;

One more Noisy is departing this world. Adieu.


When a class is deleted (either through the stack frame being reclaimed or `delete` being called), the class **destructor** is called.

Destructors are named after the class, but with a `~` in front.

If you don't specify one, the compiler makes one for you (e.g. `~Noisy() {}`)

## `const` and `class`

In [42]:
class OneNumber {
    private:
    double first;
    public:
    double getFirst() { return this->first; }
}

In [43]:
const OneNumber one;
one.getFirst()

input_line_56:2:18: error: default initialization of an object of const type 'const __cling_N545::OneNumber' without a user-provided default constructor
 const OneNumber one;
                 ^
                    {}
input_line_56:3:1: error: member function 'getFirst' not viable: 'this' argument has type 'const __cling_N545::OneNumber', but function is not marked const
one.getFirst()
^~~
input_line_55:5:12: note: 'getFirst' declared here
    double getFirst() { return this->first; }
           ^


Interpreter Error: 

What are these errors saying?

- You can't have a `const` object instance unless the class has a user-provided default constructor.
- You can't call a method on a `const` instance unless the method is marked `const`

In [50]:
class SomeNumber {
    private:
    double first;
    public:
    SomeNumber() : first(7.0) { }
    double getFirst() const { return this->first; }
}

In [51]:
const SomeNumber one;
one.getFirst()

7.0000000

In [46]:
const SomeNumber one();
one.getFirst()

input_line_60:2:22: warning: empty parentheses interpreted as a function declaration [-Wvexing-parse]
 const SomeNumber one();
                     ^~
input_line_60:2:22: note: remove parentheses to declare a variable
 const SomeNumber one();
                     ^~
input_line_60:3:1: error: base of member reference is a function; perhaps you meant to call it with no arguments?
one.getFirst()
^~~
   ()


Interpreter Error: 

To call the default constructor, **don't use parentheses!**

Just state that your object exists, and it will.

## A concession

In [52]:
class HasADouble3 {
    private:
    double cantTouchThis;
    
    public:
    HasADouble3(double const& value) : cantTouchThis(value) { }
    
    double getDouble() const {
        return this->cantTouchThis;
    }
}

In [53]:
HasADouble3* array = new HasADouble3[5];

input_line_68:2:27: error: no matching constructor for initialization of '__cling_N557::HasADouble3 [5]'
 HasADouble3* array = new HasADouble3[5];
                          ^
input_line_67:6:5: note: candidate constructor not viable: requires single argument 'value', but no arguments were provided
    HasADouble3(double const& value) : cantTouchThis(value) { }
    ^
input_line_67:1:7: note: candidate constructor (the implicit copy constructor) not viable: requires 1 argument, but 0 were provided
class HasADouble3 {
      ^
input_line_67:1:7: note: candidate constructor (the implicit move constructor) not viable: requires 1 argument, but 0 were provided


Interpreter Error: 

I can't initialize an array of things unless that thing has a default (i.e. parameterless) constructor. 

So, to store an array of things you must either:

- Make those things mutable (initialize them with dummy data, then call setters to post-initialize the values)
  - <div style="font-size: 24px">😱</div>
- Store an array of pointers
  - <div style="font-size: 24px">🙃</div>

## Class Organization

- headers and cpp files
- `#include` header
- include `.cpp` for the executable in `CMakeLists.txt`

### `book.h`

### `book.cpp`

## Copying Objects

In [2]:
class Book {
    private:
    string title;
    string author;
    int year;
    
    public:
    Book(string title, string author, int year) : title(title), author(author), year(year) {}
    
    string getTitle() const { return this->title; }
    string getAuthor() const { return this->author; }
    int getYear() const { return this->year; }
    
    string str() const {
        stringstream ss;
        ss << this->title << " by " << this->author << " (" << this->year << ")";
        return ss.str();
    }
}

In [3]:
class Bookshelf {
    private:
    Book** shelf;
    const unsigned int capacity;
    public:
    Bookshelf(unsigned int capacity) : capacity(capacity), shelf(new Book*[capacity]) {
        for (int i = 0; i < capacity; i++) {
            this->shelf[i] = nullptr;  // Why is this important?
        }
    }
    ~Bookshelf() { 
        // We only delete the shelf here. Whoever put the books on the shelf should delete the books.
        delete[] this->shelf; 
    }
    bool addBook(Book* book, unsigned int position) {
        if (position < this->capacity) {
            this->shelf[position] = book;
            cout << "Added " << book->str() << " to slot " << position << endl;
            return true;
        } else {
            return false;
        }
    }
    Book* getBook(unsigned int position) const {
        if (position < capacity) {
            return this->shelf[position];
        } else {
            return nullptr;
        }
    }
    void printBooks() const {
        for (int i = 0; i < this->capacity; i++) {
            cout << i << ": ";
            if (this->shelf[i] != nullptr) {
                cout << this->shelf[i]->str();
            }
            cout << endl;
        }
    }
}

In [4]:
Bookshelf shelf(5);
Book* book1 = new Book("The Hobbit", "JRR Tolkien", 1937);
Book* book2 = new Book("WTF", "Tim O'Reilly", 2017);
shelf.addBook(book1, 1);
shelf.addBook(book2, 4);
shelf.printBooks();

Added The Hobbit by JRR Tolkien (1937) to slot 1
Added WTF by Tim O'Reilly (2017) to slot 4
0: 
1: The Hobbit by JRR Tolkien (1937)
2: 
3: 
4: WTF by Tim O'Reilly (2017)


### Shallow Copies

In [5]:
Bookshelf otherShelf = shelf;
otherShelf.printBooks();

0: 
1: The Hobbit by JRR Tolkien (1937)
2: 
3: 
4: WTF by Tim O'Reilly (2017)


In [6]:
Book* book3 = new Book("A Short History of Nearly Everything", "Bill Bryson", 2003);

otherShelf.addBook(book3, 0);

Added A Short History of Nearly Everything by Bill Bryson (2003) to slot 0


In [7]:
shelf.printBooks();

0: A Short History of Nearly Everything by Bill Bryson (2003)
1: The Hobbit by JRR Tolkien (1937)
2: 
3: 
4: WTF by Tim O'Reilly (2017)


Why did `shelf` get modified when we added a book to `otherShelf`?

You make a copy of an object through *initialization*
```c++
Bookshelf otherShelf(shelf);
``` 
or through *assignment*
```c++
Bookshelf otherShelf = shelf;
```

If you didn't specify a **copy constructor** or an **assignment operator**, then C++ makes default ones for you.

The default versions does a simple copy of all the member variables. 


### Deep copies

In [9]:
class Book {
    private:
    string title;
    string author;
    int year;
    
    public:
    Book(string title, string author, int year) : title(title), author(author), year(year) {}
    Book(Book const& other) : title(other.title), author(other.author), year(other.year) { }
    Book & operator=(Book const& other) {
        this->title = other.title;
        this->author = other.author;
        this->year = other.year;
        return *this;
    }
    
    string getTitle() const { return this->title; }
    string getAuthor() const { return this->author; }
    int getYear() const { return this->year; }
    
    string str() const {
        stringstream ss;
        ss << this->title << " by " << this->author << " (" << this->year << ")";
        return ss.str();
    }
}

In [10]:
class Bookshelf {
    private:
    Book** shelf;
    const unsigned int capacity;
    void initShelf() {
        for (int i = 0; i < capacity; i++) {
            this->shelf[i] = nullptr;  // Why is this important?
        }
    }
    void copyOther(Bookshelf const& other) {
        for (int i = 0; i < this->capacity; i++) {
            if (other.shelf[i] != nullptr) {
                this->shelf[i] = new Book(*(other.shelf[i]));                
            }
        }
    }
    public:
    // Bookshelf shelf(7);
    Bookshelf(unsigned int const& capacity) : capacity(capacity), shelf(new Book*[capacity]) {
        this->initShelf();
    }
    // Bookshelf other(shelf);
    Bookshelf(Bookshelf const& other) : capacity(other.capacity), shelf(new Book*[other.capacity]) {
        this->initShelf();
        this->copyOther(other);
    }
    Bookshelf & operator=(Bookshelf const& other) {
        this->copyOther(other);
        return *this;
    }
    ~Bookshelf() { 
        // We only delete the shelf here. Whoever owns this shelf should delete the books first.
        delete[] this->shelf; 
    }
    bool addBook(Book* const& book, unsigned int const& position) {
        if (position < this->capacity) {
            this->shelf[position] = book;
            cout << "Added " << book->str() << " to slot " << position << endl;
            return true;
        } else {
            return false;
        }
    }
    Book* getBook(unsigned int const& position) const {
        return this->shelf[position];
    }
    void printBooks() const {
        for (int i = 0; i < this->capacity; i++) {
            cout << i << ": ";
            if (this->shelf[i] != nullptr) {
                cout << this->shelf[i]->str();
            }
            cout << endl;
        }
    }
}

In [11]:
Bookshelf shelf(5);
Book* book1 = new Book("The Hobbit", "JRR Tolkien", 1937);
Book* book2 = new Book("WTF", "Tim O'Reilly", 2017);
shelf.addBook(book1, 1);
shelf.addBook(book2, 4);
shelf.printBooks();

Added The Hobbit by JRR Tolkien (1937) to slot 1
Added WTF by Tim O'Reilly (2017) to slot 4
0: 
1: The Hobbit by JRR Tolkien (1937)
2: 
3: 
4: WTF by Tim O'Reilly (2017)


In [12]:
Bookshelf otherShelf = shelf;
Book* book3 = new Book("A Short History of Nearly Everything", "Bill Bryson", 2003);
otherShelf.addBook(book3, 3);otherShelf.printBooks();

Added A Short History of Nearly Everything by Bill Bryson (2003) to slot 3
0: 
1: The Hobbit by JRR Tolkien (1937)
2: 
3: A Short History of Nearly Everything by Bill Bryson (2003)
4: WTF by Tim O'Reilly (2017)


In [13]:
shelf.printBooks();

0: 
1: The Hobbit by JRR Tolkien (1937)
2: 
3: 
4: WTF by Tim O'Reilly (2017)


### Rule of Three

The **destructor**, **copy constructor**, and **assignment operator** are all related.

You only need them if your class dynamically allocates memory.

But if you implement one of them, you should probably implement all three.

## Key Ideas
- `struct` and `class`
  - `private` vs `public`
  - `this`
  - `->` vs `.`
  - `const`
- Class lifecycle management
  - Default constructor
  - User-defined constructor
  - Initialization list
  - Copy constructor
  - Assignment operator
  - Destructor
  - Rule of Three